In [1]:
import numpy as np
import subprocess
import time
import copy
from utils.opt_utils import *
import os
import h5py
import pickle
from qutip import *

In [2]:
wc_A = 4.069814 * (10**9) * 2 * np.pi  # cavity A frequency
wc_B = 6.096062 * (10**9) * 2 * np.pi  # cavity A frequency
wa =  5.325 * (10**9) * 2 * np.pi  # atom frequency
dt_A = np.abs(wc_A - wa) / (2 * np.pi)
dt_B = np.abs(wc_B - wa) / (2 * np.pi)
chi_A = 0.00215 * (10**9) * 2 * np.pi
chi_B = 0.00544 * (10**9) * 2 * np.pi
g_A = np.sqrt(chi_A * dt_A) * 2 * np.pi  # coupling strength w/ cavity A
g_B = np.sqrt(chi_B * dt_B) * 2 * np.pi  # coupling strength w/ cavity B

gamma = 333333.333        # atom dissipation rate
kappa_A = 10000       # cavity A dissipation rate
kappa_B = 10000       # cavity B dissipation rate

temp_q = 0.01        # avg number of thermal bath excitation for qubit
temp_A = 0.04        # avg number of thermal bath excitation for cavity A
temp_B = 0.05        # avg number of thermal bath excitation for cavity B

In [3]:
cavity_dims = 8

# Cost function
def cost_q_e(final_expect, final_dm):
    # print(final_expect[0])
    return(final_expect[0])

def cost_qA_g1(final_expect, final_state):
    # print(final_dm.full())
    return np.power(np.abs(final_state.full()[1][0]), 2)

def cost_qAB_g11(final_expect, final_dm):
    return np.power(np.abs(final_dm.full()[cavity_dims + 1][0]), 2)

def cost_qAB_g11_dm(final_expect, final_state):
    return np.power(np.abs(final_state[cavity_dims + 1][0]), 2)

def cost_qAB_g11_n(final_expect, final_dm):
    noise = (np.random.rand(1)[0] * 0.10) - 0.05
    return np.abs(final_dm.full()[cavity_dims + 1][0]) + noise

In [4]:
# ========== OPTIONS ========== #
max_segs = 20
us = 0.000001
time_start = 0 * us
time_stop = 2 * us
init_amp = 1000000
n_steps = 501

num_drives = 2
num_cavities = 1
# cavity_dims = 8
state_sizes = [2, cavity_dims]
state_vals = [0, 0]
init_freqs = [wa, wc_A]
sim_options = Options()
element_freqs = [wa, wc_A]
drive_elem_nums = [0, 1]
output_cost_func = cost_qA_g1
elements = "qA"
start_split_num = 20
n_seg_jump = 1
verbose = True
plot_trial_pulses = False

load_pulse_dir = r'C:\Users\Wang_Lab\Documents\GitLab\quantum_control_rl_server\examples\sim_1g_interp_amp'
load_times_file = r'opt_SNAP_times_1g.txt'
load_amps_file = r'opt_SNAP_amps_1g.txt'
use_loaded_data = False

save_dir = r'C:\Users\Wang_Lab\Documents\GitLab\quantum_control_rl_server\examples\sim_1g_interp_amp\save_data'
hdf5_name = time.strftime('%Y%m%d-%H%M%S.h5')
epochs = 1000
epochs_per_seg = 500
train_batch_size = 20
qubit_amp_scale = 10
cavity_amp_scale = 10
freq_scale = 0.0005
# ========== OPTIONS ========== #

t_arr = np.linspace(time_start, time_stop, n_steps)

t_step = (time_stop - time_start) / n_steps

sim_options.store_final_state = True

qscale = []
cscale = []
for i in range(2 * start_split_num):
    qscale.append(init_amp * qubit_amp_scale)
    cscale.append(init_amp * cavity_amp_scale)
    cscale.append(init_amp * cavity_amp_scale)

sm, a_A, a_B, sx, sz = reg_ops(num_cavities + 1, cavity_dims)
drive_freqs = np.array(init_freqs)

gammas = [gamma, kappa_A]
temps = [temp_q, temp_A]
c_ops = []

# Operators used in Hamiltonian
drive_ops = [sm.dag(), sm, a_A.dag(), a_A]
element_ops = [sz, a_A.dag() * a_A]
H_0 = -(chi_A * a_A.dag() * a_A * sz)
eval_ops = [sm.dag() * sm, a_A.dag() * a_A]

# Setup initial state
init_state = tensor((basis(state_sizes[0], 0) * np.sqrt(1 - state_vals[0])) + (basis(state_sizes[0], 1) * np.sqrt(state_vals[0])), (basis(state_sizes[1], 0) * np.sqrt(1 - state_vals[1])) + (basis(state_sizes[1], 1) * np.sqrt(state_vals[1])))


# Setup initial times and pulse amplitudes
times = np.linspace(time_start, time_stop, start_split_num + 2)[1:-1]

t_segs = np.array([times for i in range(2 * num_drives)])

amp_segs = (np.random.rand(2 * num_drives, start_split_num) - 0.5) * init_amp

if use_loaded_data:
    print("Loading data")
    load_pulse_times = np.loadtxt(str(os.path.join(load_pulse_dir, load_times_file)))
    load_pulse_amps = np.loadtxt(str(os.path.join(load_pulse_dir, load_amps_file)))
    
    t_segs = np.array(load_pulse_times)
    amp_segs = np.array(load_pulse_amps)
    
# print(f't_segs: {t_segs}')
# print(f'np.shape(t_segs): {np.shape(t_segs)}')
# print(f'amp_segs: {amp_segs}')
# print(f'np.shape(amp_segs): {np.shape(amp_segs)}')

# Create blank history arrays for storing optimal / past values
time_hist = []
amp_hist = []
freq_hist = []
cost_hist = []

In [ ]:
# Run vqe, etc
vmax = np.vectorize(max)
vmin = np.vectorize(min)

hdf5_start_index = 0
start_segs = start_split_num
for i in range(max_segs):
    
    temp_amp_scale = np.append(np.array(qscale), np.array(cscale))

    temp_time_scale = vmin(t_segs[:, 1:-1] - time_start, time_stop - t_segs[:, 1:-1])

    temp_freq_scale = list(freq_scale * np.array(drive_freqs))

    client_args = [num_drives, drive_ops, element_freqs, H_0, init_state, eval_ops, sim_options, output_cost_func, time_start, time_stop, drive_freqs, drive_elem_nums,verbose, plot_trial_pulses, t_segs.flatten()]
    server_args = [(len(t_segs[0]) - 2) * epochs_per_seg, train_batch_size, amp_segs, temp_amp_scale, hdf5_name]


    # Save args for rl client
    cf_name = "temp_files/client_args.txt"
    with open(cf_name, "wb") as fp:
        pickle.dump(client_args, fp)
    fp.close()

    # Save args for rl server
    sf_name = "temp_files/server_args.txt"
    with open(sf_name, "wb") as fp:
        pickle.dump(server_args, fp)
    fp.close()


    os.system('cmd /c python ./run_rl_scripts.py')

    # Give time for files to be updated etc
    time.sleep(1)

    time.sleep(10)
    opt_amps = []
    opt_times = []
    opt_freqs = []

    with h5py.File(os.path.join(save_dir, hdf5_name), "r") as f:
        opt_res_index = np.argmax(f[str(i + hdf5_start_index)]["evaluation"]["rewards"][()])
        opt_result = f[str(i + hdf5_start_index)]["evaluation"]["rewards"][()][opt_res_index]
        for j in range(2 * num_drives):
            opt_amps.append([f[str(i + hdf5_start_index)]["evaluation"]["actions"][f'pulse_array_{j}'][()][opt_res_index]])
        
    # updates amplitudes and frequencies with optimized values and reshape
    amp_segs = np.array(opt_amps)
    amp_segs = np.reshape(amp_segs, (num_drives * 2, int(len(amp_segs.flatten()) / (num_drives * 2))))

    t_segs = np.array(opt_times)
    t_segs = np.reshape(t_segs, (num_drives * 2, int(len(amp_segs.flatten()) / (num_drives * 2))))

    print(f'================')
    print(f'num segs: {i + start_segs} ')
    print(f'opt_amps: {amp_segs}')
    print(f'opt_result: {opt_result}')

    # save values to history arrays
    time_hist.append(interp_time_wrapper(t_segs, time_start, time_stop))
    amp_hist.append(interp_amp_wrapper(amp_segs))
    cost_hist.append(opt_result)

    np.savez(r'run_data\\' + hdf5_name[:-3] + "-" + str(i) + ".npz", time=time_hist[-1], amp=amp_hist[-1], freq=freq_hist[-1], cost=cost_hist[-1])

    # # Save time history
    # with h5py.File(hdf5_name, 'w'):
    #

    for i in range(2 * n_seg_jump):
        qscale.append(init_amp * qubit_amp_scale)
        cscale.append(init_amp * cavity_amp_scale)

    # split segments and return to start of loop
    if (i < max_segs - 1):
        for i in range(n_seg_jump):
            t_segs, amp_segs = split_segs_flat(interp_time_wrapper(t_segs, time_start, time_stop), interp_amp_wrapper(amp_segs))
            # amp_segs = amp_segs[:, 1:-1]